<a href="https://colab.research.google.com/github/ndb796/Deep-Learning-Paper-Review-and-Practice/blob/master/code_practices/ResNet18_CIFAR10_Train.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import time
import torch

torch.cuda.is_available()

start = time.time()

In [2]:
import torch
import random
import torch.backends.cudnn as cudnn

torch.manual_seed(0)
torch.cuda.manual_seed(0)
torch.cuda.manual_seed_all(0)
cudnn.benchmark = False
cudnn.deterministic = True
random.seed(0)

#### ResNet18 모델 정의 및 인스턴스 초기화

In [3]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.backends.cudnn as cudnn
import torch.optim as optim
import os


# ResNet18을 위해 최대한 간단히 수정한 BasicBlock 클래스 정의
class BasicBlock(nn.Module):
    def __init__(self, in_planes, planes, stride=1):
        super(BasicBlock, self).__init__()

        # 3x3 필터를 사용 (너비와 높이를 줄일 때는 stride 값 조절)
        self.conv1 = nn.Conv2d(in_planes, planes, kernel_size=3, stride=stride, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(planes) # 배치 정규화(batch normalization)

        # 3x3 필터를 사용 (패딩을 1만큼 주기 때문에 너비와 높이가 동일)
        self.conv2 = nn.Conv2d(planes, planes, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(planes) # 배치 정규화(batch normalization)

        self.shortcut = nn.Sequential() # identity인 경우
        if stride != 1: # stride가 1이 아니라면, Identity mapping이 아닌 경우
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_planes, planes, kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(planes)
            )

    def forward(self, x):
        out = F.tanh(self.bn1(self.conv1(x)))
        out = self.bn2(self.conv2(out))
        out += self.shortcut(x) # (핵심) skip connection
        out = F.tanh(out)
        return out


# ResNet 클래스 정의
class ResNet(nn.Module):
    def __init__(self, block, num_blocks, num_classes=10):
        super(ResNet, self).__init__()
        self.in_planes = 64

        # 64개의 3x3 필터(filter)를 사용
        self.conv1 = nn.Conv2d(3, 64, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(64)
        self.layer1 = self._make_layer(block, 64, num_blocks[0], stride=1)
        self.layer2 = self._make_layer(block, 128, num_blocks[1], stride=2)
        self.layer3 = self._make_layer(block, 256, num_blocks[2], stride=2)
        self.layer4 = self._make_layer(block, 512, num_blocks[3], stride=2)
        self.linear = nn.Linear(512, num_classes)

    def _make_layer(self, block, planes, num_blocks, stride):
        strides = [stride] + [1] * (num_blocks - 1)
        layers = []
        for stride in strides:
            layers.append(block(self.in_planes, planes, stride))
            self.in_planes = planes # 다음 레이어를 위해 채널 수 변경
        return nn.Sequential(*layers)

    def forward(self, x):
        out = F.tanh(self.bn1(self.conv1(x)))
        out = self.layer1(out)
        out = self.layer2(out)
        out = self.layer3(out)
        out = self.layer4(out)
        out = F.avg_pool2d(out, 4)
        out = out.view(out.size(0), -1)
        out = self.linear(out)
        return out


# ResNet18 함수 정의
def ResNet18():
    return ResNet(BasicBlock, [2, 2, 2, 2])

#### 데이터셋(Dataset) 다운로드 및 불러오기

In [4]:
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import ConcatDataset


data_transformations = {
    "crop_flip": transforms.Compose([  # 크롭 + 뒤집기
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),]), 

    "color_blur": transforms.Compose([  # 색깔 + 블러
     transforms.ColorJitter(brightness=(0.5, 0.9), 
                           contrast=(0.4, 0.8), 
                           saturation=(0.7, 0.9),
                           hue=(-0.2, 0.2),
                          ),
    transforms.GaussianBlur(kernel_size=(19, 19), sigma=(1.0, 2.0)),
    transforms.ToTensor()]),

    "basic": transforms.Compose([
    transforms.ToTensor()])
}

train_dataset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=data_transformations["basic"])
train_aug_dataset = torchvision.datasets.CIFAR10(root='./data', train=True, transform=data_transformations["crop_flip"])
train_aug_dataset2 = torchvision.datasets.CIFAR10(root='./data', train=True, transform=data_transformations["color_blur"])
                                                  
augmented_train_data = ConcatDataset((train_dataset,train_aug_dataset, train_aug_dataset2 ))

test_dataset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=data_transformations["basic"])

train_loader = torch.utils.data.DataLoader(augmented_train_data, batch_size=128, shuffle=True, num_workers=4)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=100, shuffle=False, num_workers=4)

Files already downloaded and verified
Files already downloaded and verified


#### 환경 설정 및 학습(Training) 함수 정의

In [5]:
device = 'cuda'

net = ResNet18()
net = net.to(device)
net = torch.nn.DataParallel(net)
cudnn.benchmark = True

learning_rate = 0.1
file_name = 'resnet18_cifar10.pt'

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=learning_rate, momentum=0.9, weight_decay=0.0002)


def train(epoch):
    print('\n[ Train epoch: %d ]' % epoch)
    net.train()
    train_loss = 0
    correct = 0
    total = 0
    for batch_idx, (inputs, targets) in enumerate(train_loader):
        inputs, targets = inputs.to(device), targets.to(device)
        optimizer.zero_grad()

        benign_outputs = net(inputs)
        loss = criterion(benign_outputs, targets)
        loss.backward()

        optimizer.step()
        train_loss += loss.item()
        _, predicted = benign_outputs.max(1)

        total += targets.size(0)
        correct += predicted.eq(targets).sum().item()
        
        if batch_idx % 100 == 0:
            print('\nCurrent batch:', str(batch_idx))
            print('Current benign train accuracy:', str(predicted.eq(targets).sum().item() / targets.size(0)))
            print('Current benign train loss:', loss.item())

    print('\nTotal benign train accuarcy:', 100. * correct / total)
    print('Total benign train loss:', train_loss)


def test(epoch):
    print('\n[ Test epoch: %d ]' % epoch)
    net.eval()
    loss = 0
    correct = 0
    total = 0

    for batch_idx, (inputs, targets) in enumerate(test_loader):
        inputs, targets = inputs.to(device), targets.to(device)
        total += targets.size(0)

        outputs = net(inputs)
        loss += criterion(outputs, targets).item()

        _, predicted = outputs.max(1)
        correct += predicted.eq(targets).sum().item()

    print('\nTest accuarcy:', 100. * correct / total)
    print('Test average loss:', loss / total)

    state = {
        'net': net.state_dict()
    }
    if not os.path.isdir('checkpoint'):
        os.mkdir('checkpoint')
    torch.save(state, './checkpoint/' + file_name)
    print('Model Saved!')


def adjust_learning_rate(optimizer, epoch):
    lr = learning_rate
    if epoch >= 50:
        lr = 0.01
    if epoch >= 90:
        lr = 0.001
    for param_group in optimizer.param_groups:
        param_group['lr'] = lr

#### 학습(Training) 진행

* 대략 20번의 epoch 이후에도 85%가량의 test accuracy를 얻을 수 있습니다.

In [6]:
train_start = time.time()

# for epoch in range(0, 200):
for epoch in range(0, 100):
    adjust_learning_rate(optimizer, epoch)
    train(epoch)
    test(epoch)

end = time.time()
print(f'Total: {end - start} sec')
print(f'Train: {end - train_start} sec')


[ Train epoch: 0 ]

Current batch: 0
Current benign train accuracy: 0.0703125
Current benign train loss: 2.325767755508423

Current batch: 100
Current benign train accuracy: 0.328125
Current benign train loss: 1.9956191778182983

Current batch: 200
Current benign train accuracy: 0.3671875
Current benign train loss: 1.8991990089416504

Current batch: 300
Current benign train accuracy: 0.3125
Current benign train loss: 1.9035181999206543

Current batch: 400
Current benign train accuracy: 0.3671875
Current benign train loss: 1.7265806198120117

Current batch: 500
Current benign train accuracy: 0.2578125
Current benign train loss: 1.9510366916656494

Current batch: 600
Current benign train accuracy: 0.4453125
Current benign train loss: 1.6624360084533691

Current batch: 700
Current benign train accuracy: 0.453125
Current benign train loss: 1.5394573211669922

Current batch: 800
Current benign train accuracy: 0.4375
Current benign train loss: 1.6587942838668823

Current batch: 900
Current 


Current batch: 800
Current benign train accuracy: 0.765625
Current benign train loss: 0.7654202580451965

Current batch: 900
Current benign train accuracy: 0.796875
Current benign train loss: 0.585229754447937

Current batch: 1000
Current benign train accuracy: 0.8046875
Current benign train loss: 0.7021607160568237

Current batch: 1100
Current benign train accuracy: 0.78125
Current benign train loss: 0.5611531734466553

Total benign train accuarcy: 74.98066666666666
Total benign train loss: 838.2965720593929

[ Test epoch: 5 ]

Test accuarcy: 74.77
Test average loss: 0.007209786114096641
Model Saved!

[ Train epoch: 6 ]

Current batch: 0
Current benign train accuracy: 0.796875
Current benign train loss: 0.6055938601493835

Current batch: 100
Current benign train accuracy: 0.796875
Current benign train loss: 0.5718464255332947

Current batch: 200
Current benign train accuracy: 0.828125
Current benign train loss: 0.5960429906845093

Current batch: 300
Current benign train accuracy: 0.8


Current batch: 200
Current benign train accuracy: 0.84375
Current benign train loss: 0.4364628791809082

Current batch: 300
Current benign train accuracy: 0.8515625
Current benign train loss: 0.524480402469635

Current batch: 400
Current benign train accuracy: 0.78125
Current benign train loss: 0.5998474359512329

Current batch: 500
Current benign train accuracy: 0.796875
Current benign train loss: 0.5962750315666199

Current batch: 600
Current benign train accuracy: 0.828125
Current benign train loss: 0.5066303610801697

Current batch: 700
Current benign train accuracy: 0.7734375
Current benign train loss: 0.6285082101821899

Current batch: 800
Current benign train accuracy: 0.8125
Current benign train loss: 0.5747647285461426

Current batch: 900
Current benign train accuracy: 0.8203125
Current benign train loss: 0.5306189656257629

Current batch: 1000
Current benign train accuracy: 0.7890625
Current benign train loss: 0.5854914784431458

Current batch: 1100
Current benign train accu


Current batch: 1000
Current benign train accuracy: 0.84375
Current benign train loss: 0.3905344307422638

Current batch: 1100
Current benign train accuracy: 0.859375
Current benign train loss: 0.41491520404815674

Total benign train accuarcy: 83.60733333333333
Total benign train loss: 552.7982414066792

[ Test epoch: 16 ]

Test accuarcy: 79.04
Test average loss: 0.006292220404744148
Model Saved!

[ Train epoch: 17 ]

Current batch: 0
Current benign train accuracy: 0.8828125
Current benign train loss: 0.37384748458862305

Current batch: 100
Current benign train accuracy: 0.84375
Current benign train loss: 0.4599554240703583

Current batch: 200
Current benign train accuracy: 0.8671875
Current benign train loss: 0.4381565749645233

Current batch: 300
Current benign train accuracy: 0.8515625
Current benign train loss: 0.42819368839263916

Current batch: 400
Current benign train accuracy: 0.828125
Current benign train loss: 0.4627860188484192

Current batch: 500
Current benign train accura


Current batch: 400
Current benign train accuracy: 0.859375
Current benign train loss: 0.4363941550254822

Current batch: 500
Current benign train accuracy: 0.828125
Current benign train loss: 0.4434502124786377

Current batch: 600
Current benign train accuracy: 0.84375
Current benign train loss: 0.44107353687286377

Current batch: 700
Current benign train accuracy: 0.8515625
Current benign train loss: 0.434380441904068

Current batch: 800
Current benign train accuracy: 0.828125
Current benign train loss: 0.5245715975761414

Current batch: 900
Current benign train accuracy: 0.8828125
Current benign train loss: 0.40436235070228577

Current batch: 1000
Current benign train accuracy: 0.828125
Current benign train loss: 0.4976175129413605

Current batch: 1100
Current benign train accuracy: 0.828125
Current benign train loss: 0.3915932774543762

Total benign train accuarcy: 85.13
Total benign train loss: 501.5493069291115

[ Test epoch: 22 ]

Test accuarcy: 76.6
Test average loss: 0.0079001

...


Current batch: 400
Current benign train accuracy: 0.9609375
Current benign train loss: 0.15419107675552368

Current batch: 500
Current benign train accuracy: 0.9375
Current benign train loss: 0.15182867646217346

Current batch: 600
Current benign train accuracy: 0.9375
Current benign train loss: 0.12454837560653687

Current batch: 700
Current benign train accuracy: 0.9765625
Current benign train loss: 0.11656934767961502

Current batch: 800
Current benign train accuracy: 0.96875
Current benign train loss: 0.12733696401119232

Current batch: 900
Current benign train accuracy: 0.9765625
Current benign train loss: 0.10002357512712479

Current batch: 1000
Current benign train accuracy: 0.96875
Current benign train loss: 0.10773109644651413

Current batch: 1100
Current benign train accuracy: 0.953125
Current benign train loss: 0.13979168236255646

Total benign train accuarcy: 96.67666666666666
Total benign train loss: 116.73841963894665

[ Test epoch: 72 ]

Test accuarcy: 84.82
Test averag


Current batch: 1100
Current benign train accuracy: 0.96875
Current benign train loss: 0.07934964448213577

Total benign train accuarcy: 96.61933333333333
Total benign train loss: 118.49772986210883

[ Test epoch: 77 ]

Test accuarcy: 88.41
Test average loss: 0.004100597277283668
Model Saved!

[ Train epoch: 78 ]

Current batch: 0
Current benign train accuracy: 0.984375
Current benign train loss: 0.04390101879835129

Current batch: 100
Current benign train accuracy: 0.96875
Current benign train loss: 0.08095818012952805

Current batch: 200
Current benign train accuracy: 0.9765625
Current benign train loss: 0.07642700523138046

Current batch: 300
Current benign train accuracy: 0.9921875
Current benign train loss: 0.05102047324180603

Current batch: 400
Current benign train accuracy: 0.96875
Current benign train loss: 0.1069553941488266

Current batch: 500
Current benign train accuracy: 0.9609375
Current benign train loss: 0.0953964963555336

Current batch: 600
Current benign train accur


Current batch: 500
Current benign train accuracy: 0.9765625
Current benign train loss: 0.09607671201229095

Current batch: 600
Current benign train accuracy: 0.9765625
Current benign train loss: 0.0707133561372757

Current batch: 700
Current benign train accuracy: 0.9609375
Current benign train loss: 0.08153822273015976

Current batch: 800
Current benign train accuracy: 0.96875
Current benign train loss: 0.10616406798362732

Current batch: 900
Current benign train accuracy: 0.96875
Current benign train loss: 0.08864392340183258

Current batch: 1000
Current benign train accuracy: 0.9609375
Current benign train loss: 0.14879702031612396

Current batch: 1100
Current benign train accuracy: 0.984375
Current benign train loss: 0.06436631083488464

Total benign train accuarcy: 96.53666666666666
Total benign train loss: 120.95779725536704

[ Test epoch: 83 ]

Test accuarcy: 86.85
Test average loss: 0.004803882484138012
Model Saved!

[ Train epoch: 84 ]

Current batch: 0
Current benign train a


Total benign train accuarcy: 96.52133333333333
Total benign train loss: 120.24106722138822

[ Test epoch: 88 ]

Test accuarcy: 86.17
Test average loss: 0.005592467391490936
Model Saved!

[ Train epoch: 89 ]

Current batch: 0
Current benign train accuracy: 0.9609375
Current benign train loss: 0.100846067070961

Current batch: 100
Current benign train accuracy: 0.9609375
Current benign train loss: 0.19329464435577393

Current batch: 200
Current benign train accuracy: 0.953125
Current benign train loss: 0.12529054284095764

Current batch: 300
Current benign train accuracy: 0.96875
Current benign train loss: 0.07734140753746033

Current batch: 400
Current benign train accuracy: 0.984375
Current benign train loss: 0.08244490623474121

Current batch: 500
Current benign train accuracy: 0.984375
Current benign train loss: 0.047136206179857254

Current batch: 600
Current benign train accuracy: 0.9921875
Current benign train loss: 0.04376944527029991

Current batch: 700
Current benign train acc


Current batch: 600
Current benign train accuracy: 1.0
Current benign train loss: 0.020939523354172707

Current batch: 700
Current benign train accuracy: 0.984375
Current benign train loss: 0.032324112951755524

Current batch: 800
Current benign train accuracy: 0.9921875
Current benign train loss: 0.016884230077266693

Current batch: 900
Current benign train accuracy: 0.984375
Current benign train loss: 0.026097217574715614

Current batch: 1000
Current benign train accuracy: 0.984375
Current benign train loss: 0.040960900485515594

Current batch: 1100
Current benign train accuracy: 0.9921875
Current benign train loss: 0.026844508945941925

Total benign train accuarcy: 98.776
Total benign train loss: 46.07106253900565

[ Test epoch: 94 ]

Test accuarcy: 90.85
Test average loss: 0.0033071825355291368
Model Saved!

[ Train epoch: 95 ]

Current batch: 0
Current benign train accuracy: 0.9921875
Current benign train loss: 0.02997042052447796

Current batch: 100
Current benign train accuracy: